### Data Transformation

In [1]:
import os
%pwd

'd:\\2025-courses\\mlops\\TextSummarizer\\research'

In [2]:
os.chdir('../')
%pwd

'd:\\2025-courses\\mlops\\TextSummarizer'

In [3]:
from dataclasses import dataclass
from src.textSummarizer.logging import logger
from pathlib import Path


# Parameers must be mathced with config.yaml file
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class configurationManger:
    def __init__(self,config_path=CONFIG_FILE_PATH,params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [10]:
import os
from src.textSummarizer.logging import logger

from transformers import AutoTokenizer
from datasets import load_from_disk

### Data Transformation Component

In [11]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [13]:
config=configurationManger()
data_transformation_config=config.get_data_transformation_config()
data_transformation = DataTransformation(data_transformation_config)
data_transformation.convert()

[2025-09-28 07:08:46,636: INFO:  common: yaml file: config\config.yaml loaded successfully]
[2025-09-28 07:08:46,638: INFO:  common: yaml file: params.yaml loaded successfully]
[2025-09-28 07:08:46,639: INFO:  common: created directory at: artifacts]
[2025-09-28 07:08:46,640: INFO:  common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\2025-courses\mlops\TextSummarizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 116857.65 examples/s]


## Clear Cache Options

Here are different ways to clear cache in Jupyter notebooks:

In [19]:
# 1. Clear Python __pycache__ directories
import os
import shutil

def clear_pycache():
    """Clear all __pycache__ directories in the project"""
    for root, dirs, files in os.walk('.'):
        if '__pycache__' in dirs:
            pycache_path = os.path.join(root, '__pycache__')
            print(f"Removing: {pycache_path}")
            shutil.rmtree(pycache_path)
    print("All __pycache__ directories cleared!")

# Uncomment to run:
# clear_pycache()

In [20]:
# 2. Clear Hugging Face transformers cache
from transformers import AutoTokenizer
import torch

def clear_transformers_cache():
    """Clear Hugging Face transformers cache"""
    try:
        # Clear tokenizer cache
        if hasattr(AutoTokenizer, 'clear_cache'):
            AutoTokenizer.clear_cache()
        
        # Clear PyTorch cache if using GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("GPU cache cleared!")
        
        print("Transformers cache cleared!")
    except Exception as e:
        print(f"Error clearing cache: {e}")

# Uncomment to run:
# clear_transformers_cache()

In [21]:
# 3. Restart Jupyter kernel (clears all variables and imports)
# Use: Kernel -> Restart in the menu
# Or use magic command:
# %reset -f  # This will clear all variables

print("To restart kernel: Kernel -> Restart & Clear Output")
print("To clear variables only: %reset -f")

To restart kernel: Kernel -> Restart & Clear Output
To clear variables only: %reset -f


In [22]:
# 4. Clear datasets cache
def clear_datasets_cache():
    """Clear datasets library cache"""
    try:
        from datasets import disable_caching, enable_caching
        
        # Disable caching temporarily
        disable_caching()
        print("Dataset caching disabled!")
        
        # To re-enable later:
        # enable_caching()
        
    except ImportError:
        print("datasets library not available")

# Uncomment to run:
# clear_datasets_cache()

### Quick Cache Clear Commands

**Most Common Options:**

1. **Restart Kernel**: `Kernel → Restart & Clear Output` (clears all variables)
2. **Clear Variables**: `%reset -f` (keeps imports)
3. **Clear GPU Memory**: `torch.cuda.empty_cache()` (if using GPU)
4. **Clear Python Cache**: Run the `clear_pycache()` function above

**For your TextSummarizer project**, the most useful would be:
- Restart kernel to clear memory
- Clear __pycache__ files to remove compiled Python files